<a href="https://colab.research.google.com/github/borislevant/SciComPy/blob/master/cifar10_from_softmax_to_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will demonstrate the work on the CIFAR-10 data set from the simplest Softmax model (Logistic regression) to the more advanced methods

## 1. Introduction
The CIFAR-10 dataset contains 60,000 color images of 32 x 32 pixels in 3 channels divided into 10
classes. Each class contains 6,000 images. The training set contains 50,000 images, while the test sets
provides 10,000 images. This image taken from the CIFAR repository ( <a href = "https://www.cs.toronto.edu/~kriz/cifar.html">https://www.cs.toronto.edu/~kriz/cifar.html </a>). This is a classification problem with 10 classes(muti-label classification). We can take a view on this image for more comprehension of the dataset. 

![cifar10.png](https://github.com/borislevant/SciComPy/blob/master/cifar10.png?raw=1)


The challenge is to recognize previously unseen images and assign them to one of the 10 classes.

Ok Let's get started.

## 2. Import and Preprocess the data



### 2.1 Import all required libraries

In [4]:
from __future__ import print_function
from tensorflow import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, InputLayer
from keras.layers import Conv2D, MaxPooling2D
import os

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report
import itertools

%matplotlib inline


In [5]:
batch_size = 32  # The default batch size of keras.
num_classes = 10  # Number of class for the dataset
epochs = 30
data_augmentation = False

opt = keras.optimizers.Adam(learning_rate=0.0001)
reg = keras.regularizers.l2(l2=0.0001)

### 2.2 Import and preproces of data 
We load the data and split it between train and test sets


In [6]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


170508288/170498071 [==============================] - 13s 0us/step
x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [7]:
# Normalize the data. Before we need to connvert data type to float for computation.
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices. This is called one hot encoding.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

##3. Softmax model

In [8]:
model = Sequential()
model.add(InputLayer(input_shape=x_train.shape[1:]))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax', kernel_regularizer=reg))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 10)                30730     
                                                                 
Total params: 30,730
Trainable params: 30,730
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

model.save(r'/content/drive/MyDrive/projects/model_softmax', save_format='h5')
pd.DataFrame(history.history).to_csv('/content/drive/MyDrive/projects/history_softmax.csv')


Epoch 1/30
1563/1563 [==============================] - 8s 3ms/step - loss: 1.9777 - accuracy: 0.2990 - val_loss: 1.8886 - val_accuracy: 0.3317
Epoch 2/30
1563/1563 [==============================] - 4s 3ms/step - loss: 1.8490 - accuracy: 0.3556 - val_loss: 1.8369 - val_accuracy: 0.3463
Epoch 3/30
1563/1563 [==============================] - 4s 3ms/step - loss: 1.8153 - accuracy: 0.3705 - val_loss: 1.8171 - val_accuracy: 0.3681
Epoch 4/30
1563/1563 [==============================] - 5s 3ms/step - loss: 1.7950 - accuracy: 0.3794 - val_loss: 1.7877 - val_accuracy: 0.3801
Epoch 5/30
1563/1563 [==============================] - 5s 3ms/step - loss: 1.7793 - accuracy: 0.3838 - val_loss: 1.7905 - val_accuracy: 0.3777
Epoch 6/30
1563/1563 [==============================] - 5s 3ms/step - loss: 1.7698 - accuracy: 0.3905 - val_loss: 1.7745 - val_accuracy: 0.3898
Epoch 7/30
1563/1563 [==============================] - 5s 3ms/step - loss: 1.7623 - accuracy: 0.3939 - val_loss: 1.7657 - val_accuracy:

##4. Fully-connected Neural Network (FNN) model

In [10]:
model = Sequential()
model.add(InputLayer(input_shape=x_train.shape[1:]))
model.add(Flatten())
model.add(Dense(1024, activation='relu', kernel_regularizer=reg))
model.add(Dense(256, activation='relu', kernel_regularizer=reg))
model.add(Dense(64, activation='relu', kernel_regularizer=reg))
model.add(Dense(num_classes, activation='softmax', kernel_regularizer=reg))
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dense_1 (Dense)             (None, 1024)              3146752   
                                                                 
 dense_2 (Dense)             (None, 256)               262400    
                                                                 
 dense_3 (Dense)             (None, 64)                16448     
                                                                 
 dense_4 (Dense)             (None, 10)                650       
                                                                 
Total params: 3,426,250
Trainable params: 3,426,250
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

model.save(r'/content/drive/MyDrive/projects/model_fnn', save_format='h5')
pd.DataFrame(history.history).to_csv('/content/drive/MyDrive/projects/history_fnn.csv')


Epoch 1/30
1563/1563 [==============================] - 8s 5ms/step - loss: 1.9790 - accuracy: 0.3381 - val_loss: 1.7979 - val_accuracy: 0.3974
Epoch 2/30
1563/1563 [==============================] - 7s 4ms/step - loss: 1.7534 - accuracy: 0.4076 - val_loss: 1.6873 - val_accuracy: 0.4346
Epoch 3/30
1563/1563 [==============================] - 7s 5ms/step - loss: 1.6597 - accuracy: 0.4366 - val_loss: 1.6427 - val_accuracy: 0.4386
Epoch 4/30
1563/1563 [==============================] - 7s 5ms/step - loss: 1.5939 - accuracy: 0.4563 - val_loss: 1.5857 - val_accuracy: 0.4586
Epoch 5/30
1563/1563 [==============================] - 7s 4ms/step - loss: 1.5461 - accuracy: 0.4705 - val_loss: 1.5381 - val_accuracy: 0.4798
Epoch 6/30
1563/1563 [==============================] - 7s 5ms/step - loss: 1.5097 - accuracy: 0.4845 - val_loss: 1.5385 - val_accuracy: 0.4783
Epoch 7/30
1563/1563 [==============================] - 7s 5ms/step - loss: 1.4797 - accuracy: 0.4933 - val_loss: 1.5212 - val_accuracy:

# 5. Convolutional Neural Network

In [12]:
#define the convnet
model = Sequential()
# CONV => RELU => CONV => RELU => POOL => DROPOUT
model.add(Conv2D(32, (3, 3), padding='same',input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# CONV => RELU => CONV => RELU => POOL => DROPOUT
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# FLATTERN => DENSE => RELU => DROPOUT
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# a softmax classifier
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
                                                      

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

model.save(r'/content/drive/MyDrive/projects/model_cnn', save_format='h5')
pd.DataFrame(history.history).to_csv('/content/drive/MyDrive/projects/history_cnn.csv')


Epoch 1/30
1563/1563 [==============================] - 18s 6ms/step - loss: 1.6483 - accuracy: 0.3976 - val_loss: 1.3792 - val_accuracy: 0.5011
Epoch 2/30
1563/1563 [==============================] - 9s 5ms/step - loss: 1.3807 - accuracy: 0.5019 - val_loss: 1.2451 - val_accuracy: 0.5590
Epoch 3/30
1563/1563 [==============================] - 8s 5ms/step - loss: 1.2674 - accuracy: 0.5487 - val_loss: 1.1484 - val_accuracy: 0.5870
Epoch 4/30
1563/1563 [==============================] - 8s 5ms/step - loss: 1.1862 - accuracy: 0.5812 - val_loss: 1.1117 - val_accuracy: 0.6027
Epoch 5/30
1563/1563 [==============================] - 9s 6ms/step - loss: 1.1223 - accuracy: 0.6021 - val_loss: 1.0650 - val_accuracy: 0.6240
Epoch 6/30
1563/1563 [==============================] - 10s 6ms/step - loss: 1.0699 - accuracy: 0.6220 - val_loss: 0.9800 - val_accuracy: 0.6575
Epoch 7/30
1563/1563 [==============================] - 8s 5ms/step - loss: 1.0218 - accuracy: 0.6403 - val_loss: 0.9388 - val_accurac

# 6. Transfer learning


In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16
